In [1]:
#I got this code from 'project detail' in Data Wrangling course.
#And I revised a little to fit for Python3 and my computer.
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.cElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "C:\\projects\\New_York_OSM\\New_York.osm"  # Replace this with your osm file
SAMPLE_FILE = "C:\\projects\\New_York_OSM\\New_York_sample.osm"

k = 30 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    print(root.tag, root.attrib)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n'.encode('utf-8'))
    output.write('<osm>\n  '.encode('utf-8'))

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>'.encode('utf-8'))

osm {'version': '0.6', 'generator': 'osmconvert 0.8.5', 'timestamp': '2017-09-03T15:00:02Z'}


In [1]:
#I got this schema from Data Wrangling course.
schema_contents = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    }
}

In [4]:
#I got this code from Data Wrangling course.
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus

import schema

OSM_PATH = "C:\\projects\\New_York_OSM\\New_York_sample.osm"

NODES_PATH = "C:\\projects\\New_York_OSM\\samples\\uncleaned_sample_nodes.csv"
NODE_TAGS_PATH = "C:\\projects\\New_York_OSM\\samples\\uncleaned_sample_nodes_tags.csv"
WAYS_PATH = "C:\\projects\\New_York_OSM\\samples\\uncleaned_sample_ways.csv"
WAY_NODES_PATH = "C:\\projects\\New_York_OSM\\samples\\uncleaned_sample_ways_nodes.csv"
WAY_TAGS_PATH = "C:\\projects\\New_York_OSM\\samples\\uncleaned_sample_ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema_contents

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    # YOUR CODE HERE
    if element.tag == 'node':
        for key, value in element.attrib.items():
            if key in node_attr_fields:
                if key == 'id':
                    nodeId = value
                node_attribs[key] = value
            
        for sub in element:
            key, value = sub.attrib['k'], sub.attrib['v']
            
            if not problem_chars.search(key):
                                 
                if LOWER_COLON.match(key):
                    type_value, key = key.split(":", 1)
                else:
                    type_value = default_tag_type
                    
                tags.append({"id" : nodeId, "key" : key, "value" : value, "type" : type_value})
                         
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':
        for key, value in element.attrib.items():
            if key in WAY_FIELDS:
                if key == 'id':
                    wayId = value
                way_attribs[key] = value
                
        pos = 0
        for sub in element:
            if sub.tag == 'tag':
                key, value = sub.attrib['k'], sub.attrib['v']
                
                if not problem_chars.search(key):
                   
                    if LOWER_COLON.match(key):
                        type_value, key = key.split(":", 1)
                    else:
                        type_value = default_tag_type

                    tags.append({"id" : wayId, "key" : key, "value" : value, "type" : type_value})
                        
            else:
                ref = sub.attrib['ref']
                way_nodes.append({"id" : wayId, "node_id" : ref, "position" : pos})
                pos += 1
                
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))

'''
#This part of code if for Python 2.
class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.items()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)
'''

# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with open(NODES_PATH, 'w', encoding='UTF-8', newline='') as nodes_file, \
         open(NODE_TAGS_PATH, 'w', encoding='UTF-8', newline='') as nodes_tags_file, \
         open(WAYS_PATH, 'w', encoding='UTF-8', newline='') as ways_file, \
         open(WAY_NODES_PATH, 'w', encoding='UTF-8', newline='') as way_nodes_file, \
         open(WAY_TAGS_PATH, 'w', encoding='UTF-8', newline='') as way_tags_file:

        nodes_writer = csv.DictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = csv.DictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = csv.DictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = csv.DictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = csv.DictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)

- I got some help from [this example](https://gist.github.com/carlward/54ec1c91b62a5f911c42#file-sample_project-md).

### I'm going to make the uncleaned small sample data to know the problem of my cleaning part of code.

In [1]:
#In this time, I'll designate k as 300. I'm going to check the effects of cleaning with this data.
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.cElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "C:\\projects\\New_York_OSM\\New_York.osm"  # Replace this with your osm file
SAMPLE_FILE = "C:\\projects\\New_York_OSM\\New_York_small_sample.osm"

k = 300 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    print(root.tag, root.attrib)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n'.encode('utf-8'))
    output.write('<osm>\n  '.encode('utf-8'))

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>'.encode('utf-8'))

osm {'version': '0.6', 'generator': 'osmconvert 0.8.5', 'timestamp': '2017-09-03T15:00:02Z'}


In [2]:
#In this time, I'll use cerberus and schema module to validate during writing csv file, and clean the certain fields' data during shaping.
#Use New_York_small_sample.osm and make cleaned sample csv files.
#The paths are changed. And from above shape_element function, cleaning part is added.
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus

import schema

OSM_PATH = "C:\\projects\\New_York_OSM\\New_York_small_sample.osm"

NODES_PATH = "C:\\projects\\New_York_OSM\\small_samples\\small_sample_nodes.csv"
NODE_TAGS_PATH = "C:\\projects\\New_York_OSM\\small_samples\\small_sample_nodes_tags.csv"
WAYS_PATH = "C:\\projects\\New_York_OSM\\small_samples\\small_sample_ways.csv"
WAY_NODES_PATH = "C:\\projects\\New_York_OSM\\small_samples\\small_sample_ways_nodes.csv"
WAY_TAGS_PATH = "C:\\projects\\New_York_OSM\\small_samples\\small_sample_ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema_contents

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""
    
    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    # YOUR CODE HERE
    if element.tag == 'node':

        for key, value in element.attrib.items():
            if key in node_attr_fields:
                if key == 'id':
                    nodeId = value
                node_attribs[key] = value
            
        for sub in element:
            key, value = sub.attrib['k'], sub.attrib['v']
            
            if not problem_chars.search(key):
                                        
                if LOWER_COLON.match(key):
                    type_value, key = key.split(":", 1)
                    
                else:
                    type_value = default_tag_type
                    
                if key == 'website':
                    #If the value doesn't start with 'http', I'll add 'http://'.
                    if value[:4] != 'http':
                        value = 'http://' + value

                elif key in ['postcode', 'postal_code']:
                    #I'm going to change the key name 'postal_code' to 'postcode' for consistency.
                    if key == 'postal_code':
                        key = 'postcode'
                    #If the first two digits are not numbers like 'NY', I'm going to remove them and extra spaces.
                    #And then, if the value isn't 5 digit number nor the 10 figures of zip+4 code I'll remove them.
                    if value[:2].isdigit() == False:
                        value = value[2:].strip()
                    if (len(value) == 5 and value.isdigit()) == False and \
                        (len(value) == 10 and value[:5].isdigit() and value[-4:].isdigit() and value[5] == '-') == False:
                            continue

                elif key in ['phone', 'fax', 'Fax']:
                    #I'm going to change the key name 'Fax' to 'fax' for consistency.
                    if key == 'Fax':
                        key = 'fax'
                    #I won't accept if there is any alphabet in the value.
                    if re.search(r'[a-z|A-Z]', value):
                        continue
                    #If the first two figures are '+1' or '01' or '1 ' or '1-', I'm going to remove the part.
                    if value[:2] in ['+1', '01', '1 ', '1-']:
                        value = value[2:]
                    #Temporarily leave only numbers.
                    for ch in [' ', '-', '.', '(', ')']:
                        if ch in value:
                            value = value.replace(ch, '')
                    #If the left part of value isn't 10 digits, I'll remove them.
                    if (len(value) == 10 and value.isdigit()) == False:
                        continue
                    #Change the value to follow the phone number format.
                    value = '({}) {}-{}'.format(value[:3], value[3:6], value[6:])
                        
                tags.append({"id" : nodeId, "key" : key, "value" : value, "type" : type_value})
                    
                      
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':

        for key, value in element.attrib.items():
            if key in WAY_FIELDS:
                if key == 'id':
                    wayId = value
                way_attribs[key] = value
           
        pos = 0
        for sub in element:
            if sub.tag == 'tag':
                key, value = sub.attrib['k'], sub.attrib['v']
                if not problem_chars.search(key):

                    if LOWER_COLON.match(key):
                        type_value, key = key.split(":", 1)
                        
                    else:
                        type_value = default_tag_type
                        

                    if key == 'website':
                        #If the value doesn't start with 'http', I'll add 'http://'.
                        if value[:4] != 'http':
                            value = 'http://' + value

                    elif key in ['postcode', 'postal_code']:
                        #I'm going to change the key name 'postal_code' to 'postcode' for consistency.
                        if key == 'postal_code':
                            key = 'postcode'
                        #If the first two digits are not numbers like 'NY', I'm going to remove them and extra spaces.
                        #And then, if the value isn't 5 digit number nor the 10 figures of zip+4 code I'll remove them.
                        if value[:2].isdigit() == False:
                            value = value[2:].strip()
                        if (len(value) == 5 and value.isdigit()) == False and \
                            (len(value) == 10 and value[:5].isdigit() and value[-4:].isdigit() and value[5] == '-') == False:
                                continue

                    elif key in ['phone', 'fax', 'Fax']:
                        #I'm going to change the key name 'Fax' to 'fax' for consistency.
                        if key == 'Fax':
                            key = 'fax'
                        #I won't accept if there is any alphabet in the value.
                        if re.search(r'[a-z|A-Z]', value):
                            continue
                        #If the first two figures are '+1' or '01' or '1 ' or '1-', I'm going to remove the part.
                        if value[:2] in ['+1', '01', '1 ', '1-']:
                            value = value[2:]
                        #Temporarily leave only numbers.
                        for ch in [' ', '-', '.', '(', ')']:
                            if ch in value:
                                value = value.replace(ch, '')
                        #If the left part of value isn't 10 digits, I'll remove them.
                        if (len(value) == 10 and value.isdigit()) == False:
                            continue
                        #Change the value to follow the phone number format.
                        value = '({}) {}-{}'.format(value[:3], value[3:6], value[6:])
                    
                    tags.append({"id" : wayId, "key" : key, "value" : value, "type" : type_value})
                      
            else:
                ref = sub.attrib['ref']
                way_nodes.append({"id" : wayId, "node_id" : ref, "position" : pos})
                pos += 1
                
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))

'''
#This part of code is for Python2.
class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.items()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)
'''

# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with open(NODES_PATH, 'w', encoding='UTF-8', newline='') as nodes_file, \
         open(NODE_TAGS_PATH, 'w', encoding='UTF-8', newline='') as nodes_tags_file, \
         open(WAYS_PATH, 'w', encoding='UTF-8', newline='') as ways_file, \
         open(WAY_NODES_PATH, 'w', encoding='UTF-8', newline='') as way_nodes_file, \
         open(WAY_TAGS_PATH, 'w', encoding='UTF-8', newline='') as way_tags_file:

        nodes_writer = csv.DictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = csv.DictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = csv.DictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = csv.DictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = csv.DictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)

In [3]:
#In this time, I'll use cerberus and schema module to validate during writing csv file, and without cleaning during shaping.
#Use New_York_small_sample.osm and make csv files uncleaned.
#The paths are changed. And from above shape_element function, there isn't cleaning part in here.
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus

import schema

OSM_PATH = "C:\\projects\\New_York_OSM\\New_York_small_sample.osm"

NODES_PATH = "C:\\projects\\New_York_OSM\\small_samples\\uncleaned_small_sample_nodes.csv"
NODE_TAGS_PATH = "C:\\projects\\New_York_OSM\\small_samples\\uncleaned_small_sample_nodes_tags.csv"
WAYS_PATH = "C:\\projects\\New_York_OSM\\small_samples\\uncleaned_small_sample_ways.csv"
WAY_NODES_PATH = "C:\\projects\\New_York_OSM\\small_samples\\uncleaned_small_sample_ways_nodes.csv"
WAY_TAGS_PATH = "C:\\projects\\New_York_OSM\\small_samples\\uncleaned_small_sample_ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema_contents

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    # YOUR CODE HERE
    if element.tag == 'node':
        for key, value in element.attrib.items():
            if key in node_attr_fields:
                if key == 'id':
                    nodeId = value
                node_attribs[key] = value
            
        for sub in element:
            key, value = sub.attrib['k'], sub.attrib['v']
            
            if not problem_chars.search(key):
                                 
                if LOWER_COLON.match(key):
                    type_value, key = key.split(":", 1)
                else:
                    type_value = default_tag_type
                    
                tags.append({"id" : nodeId, "key" : key, "value" : value, "type" : type_value})
                         
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':
        for key, value in element.attrib.items():
            if key in WAY_FIELDS:
                if key == 'id':
                    wayId = value
                way_attribs[key] = value
                
        pos = 0
        for sub in element:
            if sub.tag == 'tag':
                key, value = sub.attrib['k'], sub.attrib['v']
                
                if not problem_chars.search(key):
                   
                    if LOWER_COLON.match(key):
                        type_value, key = key.split(":", 1)
                    else:
                        type_value = default_tag_type

                    tags.append({"id" : wayId, "key" : key, "value" : value, "type" : type_value})
                        
            else:
                ref = sub.attrib['ref']
                way_nodes.append({"id" : wayId, "node_id" : ref, "position" : pos})
                pos += 1
                
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))

'''
#This part of code is for Python2.
class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.items()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)
'''

# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with open(NODES_PATH, 'w', encoding='UTF-8', newline='') as nodes_file, \
         open(NODE_TAGS_PATH, 'w', encoding='UTF-8', newline='') as nodes_tags_file, \
         open(WAYS_PATH, 'w', encoding='UTF-8', newline='') as ways_file, \
         open(WAY_NODES_PATH, 'w', encoding='UTF-8', newline='') as way_nodes_file, \
         open(WAY_TAGS_PATH, 'w', encoding='UTF-8', newline='') as way_tags_file:

        nodes_writer = csv.DictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = csv.DictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = csv.DictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = csv.DictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = csv.DictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)

### After auditing the data, I found that the above cleaning code works.

### <dr> Therefore in this time, I'm going to make sample csv files and audit them.

In [3]:
#In this time, I'll use cerberus and schema module to validate during writing csv file, and clean the certain fields' data during shaping.
#Use New_York_sample.osm and make cleaned sample csv files.
#The paths are changed. And from above shape_element function, cleaning part is added.
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus

import schema

OSM_PATH = "C:\\projects\\New_York_OSM\\New_York_sample.osm"

NODES_PATH = "C:\\projects\\New_York_OSM\\samples\\sample_nodes.csv"
NODE_TAGS_PATH = "C:\\projects\\New_York_OSM\\samples\\sample_nodes_tags.csv"
WAYS_PATH = "C:\\projects\\New_York_OSM\\samples\\sample_ways.csv"
WAY_NODES_PATH = "C:\\projects\\New_York_OSM\\samples\\sample_ways_nodes.csv"
WAY_TAGS_PATH = "C:\\projects\\New_York_OSM\\samples\\sample_ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema_contents

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""
    
    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    # YOUR CODE HERE
    if element.tag == 'node':

        for key, value in element.attrib.items():
            if key in node_attr_fields:
                if key == 'id':
                    nodeId = value
                node_attribs[key] = value
            
        for sub in element:
            key, value = sub.attrib['k'], sub.attrib['v']
            
            if not problem_chars.search(key):
                                        
                if LOWER_COLON.match(key):
                    type_value, key = key.split(":", 1)
                    
                else:
                    type_value = default_tag_type
                

                if key == 'website':
                    #If the value doesn't start with 'http', I'll add 'http://'.
                    if value[:4] != 'http':
                        value = 'http://' + value

                elif key in ['postcode', 'postal_code']:
                    #I'm going to change the key name 'postal_code' to 'postcode' for consistency.
                    if key == 'postal_code':
                        key = 'postcode'
                    #If the first two digits are not numbers like 'NY', I'm going to remove them and extra spaces.
                    #And then, if the value isn't 5 digit number nor the 10 figures of zip+4 code I'll remove them.
                    if value[:2].isdigit() == False:
                        value = value[2:].strip()
                    if (len(value) == 5 and value.isdigit()) == False and \
                        (len(value) == 10 and value[:5].isdigit() and value[-4:].isdigit() and value[5] == '-') == False:
                            continue

                elif key in ['phone', 'fax', 'Fax']:
                    #I'm going to change the key name 'Fax' to 'fax' for consistency.
                    if key == 'Fax':
                        key = 'fax'
                    #I won't accept if there is any alphabet in the value.
                    if re.search(r'[a-z|A-Z]', value):
                        continue
                    #If the first two figures are '+1' or '01' or '1 ' or '1-', I'm going to remove the part.
                    if value[:2] in ['+1', '01', '1 ', '1-']:
                        value = value[2:]
                    #Temporarily leave only numbers.
                    for ch in [' ', '-', '.', '(', ')']:
                        if ch in value:
                            value = value.replace(ch, '')
                    #If the left part of value isn't 10 digits, I'll remove them.
                    if (len(value) == 10 and value.isdigit()) == False:
                        continue
                    #Change the value to follow the phone number format.
                    value = '({}) {}-{}'.format(value[:3], value[3:6], value[6:])
                        
                tags.append({"id" : nodeId, "key" : key, "value" : value, "type" : type_value})
                    
                      
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':

        for key, value in element.attrib.items():
            if key in WAY_FIELDS:
                if key == 'id':
                    wayId = value
                way_attribs[key] = value
           
        pos = 0
        for sub in element:
            if sub.tag == 'tag':
                key, value = sub.attrib['k'], sub.attrib['v']
                if not problem_chars.search(key):

                    if LOWER_COLON.match(key):
                        type_value, key = key.split(":", 1)
                        
                    else:
                        type_value = default_tag_type
                      

                    if key == 'website':
                        #If the value doesn't start with 'http', I'll add 'http://'.
                        if value[:4] != 'http':
                            value = 'http://' + value

                    elif key in ['postcode', 'postal_code']:
                        #I'm going to change the key name 'postal_code' to 'postcode' for consistency.
                        if key == 'postal_code':
                            key = 'postcode'
                        #If the first two digits are not numbers like 'NY', I'm going to remove them and extra spaces.
                        #And then, if the value isn't 5 digit number nor the 10 figures of zip+4 code I'll remove them.
                        if value[:2].isdigit() == False:
                            value = value[2:].strip()
                        if (len(value) == 5 and value.isdigit()) == False and \
                            (len(value) == 10 and value[:5].isdigit() and value[-4:].isdigit() and value[5] == '-') == False:
                                continue

                    elif key in ['phone', 'fax', 'Fax']:
                        #I'm going to change the key name 'Fax' to 'fax' for consistency.
                        if key == 'Fax':
                            key = 'fax'
                        #I won't accept if there is any alphabet in the value.
                        if re.search(r'[a-z|A-Z]', value):
                            continue
                        #If the first two figures are '+1' or '01' or '1 ' or '1-', I'm going to remove the part.
                        if value[:2] in ['+1', '01', '1 ', '1-']:
                            value = value[2:]
                        #Temporarily leave only numbers.
                        for ch in [' ', '-', '.', '(', ')']:
                            if ch in value:
                                value = value.replace(ch, '')
                        #If the left part of value isn't 10 digits, I'll remove them.
                        if (len(value) == 10 and value.isdigit()) == False:
                            continue
                        #Change the value to follow the phone number format.
                        value = '({}) {}-{}'.format(value[:3], value[3:6], value[6:])
                    
                    tags.append({"id" : wayId, "key" : key, "value" : value, "type" : type_value})
                      
            else:
                ref = sub.attrib['ref']
                way_nodes.append({"id" : wayId, "node_id" : ref, "position" : pos})
                pos += 1
                
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))

'''
#This part of code is for Python2.
class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.items()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)
'''

# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with open(NODES_PATH, 'w', encoding='UTF-8', newline='') as nodes_file, \
         open(NODE_TAGS_PATH, 'w', encoding='UTF-8', newline='') as nodes_tags_file, \
         open(WAYS_PATH, 'w', encoding='UTF-8', newline='') as ways_file, \
         open(WAY_NODES_PATH, 'w', encoding='UTF-8', newline='') as way_nodes_file, \
         open(WAY_TAGS_PATH, 'w', encoding='UTF-8', newline='') as way_tags_file:

        nodes_writer = csv.DictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = csv.DictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = csv.DictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = csv.DictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = csv.DictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)